In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import (
    HumanMessage, 
    SystemMessage, 
    ToolMessage
)
from langchain.tools import tool
from langchain_core.output_parsers.openai_tools import parse_tool_calls
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("API_KEY")
base_url = os.getenv("OPENAI_ENDPOINT")
model_name = "gpt-4o-mini"
temp=0.0

llm = ChatOpenAI(
    base_url=base_url,
    api_key=api_key,
    model=model_name,
    temperature=temp
)

**Tool creation**

In [ ]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [ ]:
tools = [multiply]
tool_map = {tool.name:tool for tool in tools}

In [ ]:
tool_map

**Binding Tools**

In [ ]:
llm_with_tools = llm.bind_tools(tools)

In [ ]:
question = "3 multiplied by 2"

In [ ]:
messages = [
    SystemMessage("You're a helpful assistant"),
    HumanMessage(question)
]

In [ ]:
ai_message = llm_with_tools.invoke(messages)

In [ ]:
ai_message

In [ ]:
messages.append(ai_message)

In [ ]:
messages

**Using Tool Calls**

In [ ]:
parsed_tool_calls = parse_tool_calls(
    ai_message.additional_kwargs.get("tool_calls")
)

In [ ]:
parsed_tool_calls

In [ ]:
for tool_call in parsed_tool_calls:
    tool_call_id = tool_call['id']
    function_name = tool_call['name']
    arguments = tool_call['args']
    func = tool_map[function_name]
    result = func.invoke(arguments)
    tool_message = ToolMessage(
        content=result,
        name=function_name,
        tool_call_id=tool_call_id,
    )
    messages.append(tool_message)

**Sending the result back to the LLM**

In [ ]:
messages

In [ ]:
ai_message = llm_with_tools.invoke(messages)

In [ ]:
ai_message